> 본 커널은 [XGBoost Starter - LB 0.793](https://www.kaggle.com/code/cdeotte/xgboost-starter-0-793)에 작성된 코드를 기반으로 한국어 설명을 추가한 XGBoost 튜토리얼입니다.

> TOC
```
Step 1. Load Libraries
Step 2. Load Dataset
Step 3. Feature Engineering
Step 4. Train XGB
Step 5. Save OOF Preds
Step 6. Feature Importances
Step 7. Process and Feature Engineer Test Data
Step 8. Infer Test
Step 9. Create Submission CSV
```

## Step 1. Load Libraries

우리가 머신러닝, 딥러닝을 학습시킬 때 CPU를 사용하여 데이터 전처리 등을 수행하고 GPU로 모델 학습을 하는 것이 일반적입니다. 그리고 이 때는 CPU에 올라간 데이터를 GPU로 복사(이동)하는 과정이 필요합니다. pandas로 데이터프레임을 다루고 torch로 GPU 메모리 상의 데이터를 처리하는 식이죠.

이제는 그러지 말고, '전체 과정을 모두 GPU 위에서 진행하자' 라는 컨셉으로 나온게 RAPIDS입니다. RAPIDS는 엔비디아가 주도하여 구축하고 운영하고 있는 CUDA 프로세스 기반 데이터사이언스 플랫폼입니다.

CUDA를 기반으로 한다는 것을 강조하듯 패키지 이름도 모두 cuxx로 지었습니다. pandas는 cudf로, numpy는 cupy로, sklearn은 cuml로 대체하여 거의 기존과 동일한 함수를 사용할 수 있게 개발되어 있습니다.

In [1]:
import pandas as pd
import numpy as np

import cupy
import cudf

import matplotlib.pyplot as plt, gc, os

print('cudf version',cudf.__version__)

## Step 2. Load Dataset

cudf를 사용해볼 것입니다. 기본적인 함수나 사용법은 pandas와 동일합니다.

In [2]:
# read_parquet() 함수는 parquet 형식의 파일을 읽어옵니다.
df = cudf.read_parquet('../input/amex-data-integer-dtypes-parquet-format/train.parquet')

In [3]:
df

`customer_ID`를 숫자로 깔끔하게 변환해주고 싶다면, hex_to_int() 함수를 사용하고 astype()을 통해 처리할 수 있습니다.
여기서는 마지막 16문자에 대해서 해당 처리를 수행합니다.

In [4]:
df['customer_ID'] = df['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
df

`S_2`열은 시간 정보를 담고 있습니다. pandas와 마찬가지로 to_datetime()함수로 데이터 타입을 변경해줍니다.

In [5]:
df['S_2'] = cudf.to_datetime(df['S_2'])
df

빈 셀도 있으니 fillna() 함수로 채우는데, 1byte(8bit)중 가장 낮은 숫자인 -127을 넣어줄 것입니다.

In [6]:
df.isna().sum()

In [7]:
NAN_VALUE = -127 
df = df.fillna(NAN_VALUE)
df.isna().sum()

아래 함수는 이 과정을 한번에 수행해줍니다.

In [8]:
def read_file(path = '', usecols = None):
    # read_parquet() 함수는 parquet 형식의 파일을 읽어옵니다.
    # 이 때, 컬럼을 지정해주고 싶다면
    if usecols is not None: 
        df = cudf.read_parquet(path, columns=usecols)
    # columns를 지정하지 않고 그대로 불러온다면,
    else: df = cudf.read_parquet(path)
    
    df['customer_ID'] = df['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
    df.S_2 = cudf.to_datetime( df.S_2 )
    df = df.fillna(NAN_VALUE) 
    print('shape of data:', df.shape)
    
    return df

# print('Reading train data...')
# TRAIN_PATH = '../input/amex-data-integer-dtypes-parquet-format/train.parquet'
# train = read_file(path = TRAIN_PATH)

원본 커널에서는 df가 아닌 train 변수명을 사용했습니다. 같은 변수 명을 유지하기 위해 복사해줍니다.

단, 여기서 copy()함수를 사용하지 않습니다. 데이터 사이즈가 크기때문에 copy()함수를 사용하면 메모리에 부담을 줄 수 있으니 메모리 참조 방식을 의도적으로 사용합니다. 

In [9]:
train = df
train.head()

## Step 3. Feature Engineering

이번에는 원본 커널에서 사용하는 함수를 그대로 사용할 것입니다. 데이터 집계 및 병합하는 과정에서 multiindex를 1차원으로 묶는 부분만 미리 살펴봅니다.

In [10]:
train.info()

In [11]:
multi_index_col_sample = train.groupby('customer_ID')[['B_30','B_38','D_114']].agg(['count','last','nunique']).columns
multi_index_col_sample

In [12]:
['_'.join(x) for x in multi_index_col_sample]

이렇게 이중 인덱스를 1차원 인덱스로 붙여서 보기 쉽게 만들어줄 수 있습니다. 이어서 전체 함수를 확인합니다.

In [13]:
def process_and_feature_engineer(df):
    # list comprehension 방식으로 customer_ID 컬럼과 S_2 컬럼을 제외한 나머지 컬럼명을 all_cols 변수에 담아줍니다. 
    # 해당 컬럼들은 인덱스와 ID값이 아닌 분석 대상 데이터를 가지고 있습니다.
    all_cols = [c for c in list(df.columns) if c not in ['customer_ID','S_2']]
    
    # all_cols 중에서 카테고리형 변수와 수치형 변수를 나눠줍니다.
    # 원본 커널에서 카테고리형 변수는 아래 11개 컬럼만 지정하고 있으나, 더 많은 컬럼이 있는 것으로 보입니다.
    # 하지만 연속성을 유지하고 혼선을 방지하기 위해 그대로 사용하도록 하겠습니다.
    cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
    num_features = [col for col in all_cols if col not in cat_features]

    # 각 customer_ID에 대해 수치형 변수들을 통계치로 집계해줍니다.
    test_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    # 집계한 컬럼 명은 MultiIndex 타입입니다. '_' 문자로 컬럼명을 이어줍니다.
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]

    # 각 customer_ID에 대해 카테고리형 변수를 집계해줍니다.
    # count는 동일한 customer_ID가 몇번 등장하는지, last는 해당 customer_ID의 각 카테고리형 변수 중 가장 최근 값이 무엇인지 보여줍니다.
    # nunique()는 각 카테고리형 변수에 등장하는 유일한 값을 세어줍니다.
    test_cat_agg = df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]

    # 기존 데이터와 통계치를 구한 값을 모두 병합해줍니다.
    df = cudf.concat([test_num_agg, test_cat_agg], axis=1)
    del test_num_agg, test_cat_agg
    print('shape after engineering', df.shape )
    
    return df


train = process_and_feature_engineer(train)

In [14]:
train

데이터셋은 타겟(label) 데이터도 제공하고 있습니다. 위에서 생성한 train 데이터셋에 병합해주겠습니다.
이 때, customer_ID는 앞에서 했던 것과 동일한 방식으로 정수형 처리를 해줘야 합니다. 본 테스크는 해당 customer가 대출을 갚을 것인가, 갚지 못할 것인가를 예측하는 과제입니다. customer 마다의 대출 상환 여부가 train_labels.csv에 담겨있습니다.

In [15]:
targets = cudf.read_csv('../input/amex-default-prediction/train_labels.csv')
targets['customer_ID'] = targets['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
targets = targets.set_index('customer_ID')

# 인덱스는 customer_ID로 동일합니다. 해당 인덱스를 기준으로 merge합니다.
train = train.merge(targets, left_index=True, right_index=True, how='left')
# 타겟 데이터는 8비트 정수형으로 담아줍니다.
train.target = train.target.astype('int8')
# targets는 이제 train데이터셋에 병합되었으므로 메모리 상에서 제거해줍니다.
del targets

In [16]:
train = train.reset_index()
train

모델에 학습시킬 피처 수를 세어봅니다. 첫번째 열은 id값이고 마지막 열은 label입니다. 2개 열을 제외한 나머지 열의 수는 198개입니다.

In [17]:
FEATURES = train.columns[1:-1]
print(f'There are {len(FEATURES)} features!')

## Step 4. Train XGB
모델을 학습할 때는 KFold 교차검증을 활용해 모든 데이터를 최소 1회 이상 학습에 사용하면서 과적합을 막을 것입니다.
일반적으로 편의상 사용되는 70:30 혹은 80:20 등으로 데이터를 학습/검증 용으로 나눠 학습시키는 방식은 30, 20에 해당하는 부분은 학습에 활용할 수 없다는 문제가 있습니다. KFold는 학습데이터와 검증데이터를 쪼개서(K개의 Fold를 만든다고 표현합니다) 모든 데이터셋이 학습에 활용될 수 있도록 합니다.

In [18]:
# LOAD XGB LIBRARY
from sklearn.model_selection import KFold
import xgboost as xgb
print('XGB Version',xgb.__version__)

# XGB MODEL PARAMETERS
xgb_parms = { 
    'max_depth':4, 
    'learning_rate':0.05, 
    'subsample':0.8,
    'colsample_bytree':0.6, 
    'eval_metric':'logloss',
    'objective':'binary:logistic',
    'tree_method':'gpu_hist',
    'predictor':'gpu_predictor',
    'random_state':42
}

학습할 때, DeviceQuantileDMatrix를 사용합니다. GPU는 기본적으로 한번 사용시 모든 메모리가 일시에 연산에 사용되는데, 해당 함수를 사용하면 
GPU 메모리를 작은 단위로 분할해서 사용하게 해줍니다. 

DeviceQuantileDMatrix를 사용하기 위해서는 iteration 방식, 즉 next() 함수를 반복 호출하며 배치 형태로 데이터를 넘겨줄 수 있는 클래스를 정의해줘야 합니다.

In [19]:
class IterLoadForDMatrix(xgb.core.DataIter):
    def __init__(self, df=None, features=None, target=None, batch_size=256*1024):
        self.features = features
        self.target = target
        self.df = df
        # 0번부터 시작해 데이터를 모두 넘겨줄때까지 1씩 올려줄 것입니다.
        self.it = 0 
        self.batch_size = batch_size
        # np.ceil()은 '올림'을 해주는 함수입니다. 
        # 데이터를 배치 사이즈로 나눠 배치 크기(갯수)를 계산합니다.
        self.batches = int( np.ceil( len(df) / self.batch_size ) )
        super().__init__()

    def reset(self):
        '''Reset the iterator'''
        # iteration을 처음부터 다시 실시해줘야 한다면 reset() 함수로 초기화할 수 있습니다.
        self.it = 0

    def next(self, input_data):
        '''Yield next batch of data.'''
        # 클래스 인스턴스 생성시 self.batches가 정의되었습니다. 전달할 수 있는 총 배치 수를 담고 있습니다.
        # self.it이 전달 가능한 배치 수에 도달했을 때, iteration을 종료합니다.
        if self.it == self.batches:
            # 
            return 0 
        
        # 인덱싱을 위한 start-end point를 구합니다.
        a = self.it * self.batch_size
        b = min( (self.it + 1) * self.batch_size, len(self.df) )
        # 배치로 전달할 데이터를 인덱싱해서 dt 변수에 담아줍니다.
        dt = cudf.DataFrame(self.df.iloc[a:b])
        # dt로 받은 데이터에서 피처와 타겟을 넘겨줍니다.
        input_data(data=dt[self.features], label=dt[self.target]) #, weight=dt['weight'])
        
        # iter를 1씩 올려주면서 다음 배치를 수행하기 위한 계산입니다.
        self.it += 1
        return 1

아래 함수는 본 데이터를 공유한 [American Express - Default Prediction](https://www.kaggle.com/competitions/amex-default-prediction) 대회에서 모델을 평가하는 로직입니다.([원문](https://www.kaggle.com/competitions/amex-default-prediction/discussion/327534))

모델 학습 과정에서 최적화하는 기준으로 사용하게 됩니다.




In [20]:
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

드디어 모델을 학습시킵니다. 제한된 GPU를 효율적으로 사용하기 위해 데이터셋을 CPU로 먼저 내려줄 것입니다. 사실, RAPIDS 플랫폼을 사용한다면 모든 과정을 GPU로 수행하는 것이 효과적이지만 아직 온전하게 적용되기에는 어려운 것 같습니다.

아무튼, to_pandas() 함수를 사용하면 pandas dataframe 객체로 들어가면서 CPU 자원으로 복사할 수 있습니다.

In [21]:
train = train.to_pandas()

이렇게 했을 때, 여전히 GPU상에서 여러차례 동일한 데이터셋이 저장된 메모리를 참조한 변수들이 있을 것입니다. 그러한 변수들을 깔끔하게 제거해줘야 원활한 GPU 사용이 가능할 것입니다.

파이썬은 내부적으로 Garbage Collection이 생성된 객체를 순회하며 '사용되지 않는 객체'를 자동으로 할당 해제 해주는 프로세스를 가지고 있습니다.
해당 프로세스가 '사용되지 않는 객체'를 현재 시점에서 인식할 수 있도록 gc.collecter()를 실행해줍니다. 실행하면 메모리 해제된 객체 숫자를 반환해줍니다.

In [22]:
gc.collect()

KFold의 K는 5개로 지정할 것입니다. 그러면 5개의 Fold에 대해 4개의 학습 폴드, 1개의 검증 폴드를 가지게 되고, 검증 폴드의 위치를 옮겨가며 총 5회 반복 학습을 시행합니다. 결과적으로 5번의 검증 결과에 대한 평균값을 통해 최적화를 해나가게 됩니다.

SEED는 임의의 숫자로 지정하면 됩니다.

In [23]:
FOLDS = 5
SEED = 42
skf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

In [25]:
importances = []
oof = []
TRAIN_SUBSAMPLE = 1.0
VER = 1 # 모델을 저장할 때 기록할 버전 정보입니다.

# KFold 객체 skf에 대해 교차검증을 수행하기 위하여 학습, 검증 폴드로 분리한 다음 순회하며 반복 검증을 실시합니다.
for fold,(train_idx, valid_idx) in enumerate(skf.split(train, train.target)):
    
    # train data 중에서도 일부만 샘플링해서 학습을 수행하고 싶다면 TRAIN_SUBSAMPLE을 1 미만으로 설정해줍니다.
    # 그럼 아래 if문을 통해 그 비율만큼 랜덤으로 추출해서 다시 train set을 구성할 수 있습니다.
    if TRAIN_SUBSAMPLE<1.0:
        np.random.seed(SEED)
        train_idx = np.random.choice(train_idx, int(len(train_idx)*TRAIN_SUBSAMPLE), replace=False)
        np.random.seed(None)
    
    print('#'*25)
    print('### Fold',fold+1)
    print('### Train size',len(train_idx),'Valid size',len(valid_idx))
    print(f'### Training with {int(TRAIN_SUBSAMPLE*100)}% fold data...')
    print('#'*25)
    
    # 앞에서 만들어둔 IterLoadForDMatrix 객체 인스턴스를 생성해줍니다. 배치로 던져주며 학습하기 위함입니다.
    Xy_train = IterLoadForDMatrix(train.loc[train_idx], FEATURES, 'target')
    
    # 학습을 수행하면서 성능을 검증해줄 검증 데이터도 정의해줍니다.
    X_valid = train.loc[valid_idx, FEATURES]
    y_valid = train.loc[valid_idx, 'target']
    
    # 이제 DeviceQuantileDMatrix를 사용해 iteration을 돌면서 학습할 수 있는 dtrain 객체를 만들어주고,
    dtrain = xgb.DeviceQuantileDMatrix(Xy_train, max_bin=256)
    # 마찬가지로 train 데이터와 같은 형태를 취해 비교연산이 가능하도록 DMatrix로 처리해줍니다.
    dvalid = xgb.DMatrix(data=X_valid, label=y_valid)
    
    # 모델을 학습합니다.
    model = xgb.train(xgb_parms, 
                dtrain=dtrain,
                evals=[(dtrain,'train'),(dvalid,'valid')],
                num_boost_round=9999,
                early_stopping_rounds=100,
                verbose_eval=100) 
    # 교차 검증 중 1회 검증시마다 모델을 저장해줍니다.
    model.save_model(f'XGB_v{VER}_fold{fold}.xgb')
    
    # 모델 학습이 끝나고 feature importance를 확인할 것입니다. 이를 위해 매 학습마다 importance를 계산하여 변수에 저장해줍니다.
    dd = model.get_score(importance_type='weight')
    df = pd.DataFrame({'feature':dd.keys(),f'importance_{fold}':dd.values()})
    importances.append(df)
            
    # 모델을 검증합니다. 정확도는 위에서 함수로 만들어둔 대회 평가지표를 사용합니다.
    oof_preds = model.predict(dvalid)
    acc = amex_metric_mod(y_valid.values, oof_preds)
    print('Kaggle Metric =',acc,'\n')
    
    # 검증 스코어(oof_pred)도 따로 저장해줍니다.
    df = train.loc[valid_idx, ['customer_ID','target'] ].copy()
    df['oof_pred'] = oof_preds
    oof.append( df )
    
    # 학습을 위해 사용한 변수들은 모두 메모리 해제해줍니다.
    del dtrain, Xy_train, dd, df
    del X_valid, y_valid, dvalid, model
    # 메모리 참조된 것들도 완전히 제거하기 위해 가비지 컬랙션을 실시해줍니다.
    _ = gc.collect()
    
print('#'*25)
# 학습이 종료되면 전체 검증 결과를 데이터프레임으로 저장해주고, 
# 실제 값과 검증 값을 평가지표로 계산해줍니다.
oof = pd.concat(oof,axis=0,ignore_index=True).set_index('customer_ID')
acc = amex_metric_mod(oof.target.values, oof.oof_pred.values)
print('OVERALL CV Kaggle Metric =',acc)

In [26]:
# 학습이 종료되었으니 이제 데이터셋도 필요가 없어졌습니다.
# 메모리를 한번 더 정리해주겠습니다.
del train
_ = gc.collect()

## Step 5. Save OOF Preds

예측 결과를 customer_ID와 함께 저장해줄 것입니다. 데이터 파일에서 unique한 id 정보만 가져와 앞에서 수행했던 것처럼 16진수를 정수형으로 바꿔주고, 학습 과정에서 확보한 예측값을 병합해주면 됩니다.

In [28]:
TRAIN_PATH = '../input/amex-data-integer-dtypes-parquet-format/train.parquet'
oof_xgb = pd.read_parquet(TRAIN_PATH, columns=['customer_ID']).drop_duplicates()
oof_xgb['customer_ID_hash'] = oof_xgb['customer_ID'].apply(lambda x: int(x[-16:],16) ).astype('int64')
oof_xgb = oof_xgb.set_index('customer_ID_hash')
oof_xgb = oof_xgb.merge(oof, left_index=True, right_index=True)
oof_xgb = oof_xgb.sort_index().reset_index(drop=True)
oof_xgb.to_csv(f'oof_xgb_v{VER}.csv',index=False)
oof_xgb.head()

예측 결과를 시각화해봅니다. 예측 결과는 0에서 1사이의 확률입니다. 단 5%만 채무 불이행(default, 1)이고 나머지는 0으로 예측해야 하기 때문에 시각화했을 때 0과 1쪽에 양방향으로 쏠려있되, 0에 더 많은 값이 분포되어 있어야 합니다.

In [29]:
plt.hist(oof_xgb.oof_pred.values, bins=100)
plt.title('OOF Predictions')
plt.show()

예측치를 csv 파일로 저장했으니 변수를 제거합니다. 이렇게 파일로 저장하고 메모리를 비우는 작업을 반복하는 이유는 캐글 상에서 지원되는 RAM이 크지 않기 때문입니다. 일반적으로 로컬 환경에서도 RAM은 여유롭지 않기 때문에 중간에 참조나 복사를 하면서 셧다운 되는 일이 발생할 수 있는데, 이를 방지해주기 위해 하드디스크로 파일을 저장하고 RAM은 가벼운 상태를 유지해주는 것이 좋습니다.

In [30]:
del oof_xgb, oof
_ = gc.collect()

## Step 6. Feature Importance

Feature Importance는 피처 중요도, 즉 모델이 테스크를 수행(여기서는 예측)하는데에 어떤 변수가 활용 비중이 높은가에 대한 정보입니다.
모델 학습이 끝난 다음 이렇게 살펴보면서 예측에 중요하지 않은 변수가 있다면 제거하고 중요도가 지나치게 변수들은 예측 대상과의 인과성 혹은 상관성을 확인하는 등의 피드백 과정을 거치게 됩니다.

In [39]:
import matplotlib.pyplot as plt

# 교차검증을 수행하면서 FOLD 수(5)만큼 importances도 누적해서 구해졌을 것입니다.
# df 변수에 병합하여 feature마다 평균 importance를 계산해줍니다.
df = importances[0].copy()
for k in range(1,FOLDS):
    df = df.merge(importances[k], on='feature', how='left')
df['importance'] = df.iloc[:,1:].mean(axis=1)
df = df.sort_values('importance',ascending=False)
df.to_csv(f'xgb_feature_importance_v{VER}.csv',index=False)

In [40]:
df

bar 차트로 중요도 기준 상위 20개 feature만 시각화해줍니다.

In [41]:
NUM_FEATURES = 20
plt.figure(figsize=(10,5*NUM_FEATURES//10))
plt.barh(np.arange(NUM_FEATURES,0,-1), df.importance.values[:NUM_FEATURES])
plt.yticks(np.arange(NUM_FEATURES,0,-1), df.feature.values[:NUM_FEATURES])
plt.title(f'XGB Feature Importance - Top {NUM_FEATURES}')
plt.show()

## Step 7. Process and Feature Engineer Test Data

이제 위에서 학습한 xgboost 모델로 테스트 데이터를 예측합니다. 먼저 테스트 데이터를 살펴보겠습니다.

In [42]:
TEST_PATH = '../input/amex-data-integer-dtypes-parquet-format/test.parquet'
test = read_file(path = TEST_PATH, usecols = ['customer_ID','S_2'])
test

테스트 데이터 역시 사이즈가 크기 때문에 램에 한번에 올리는 것은 무리가 있겠습니다. 데이터셋을 분할해서 넣어줄 수 있도록 처리해주기 위해 함수를 만들어 주겠습니다.

In [56]:
# 4개 PART로 분리해주는 함수입니다.
def get_rows(customers, test, NUM_PARTS = 4, verbose = ''):
    # 한 청크(1개 PART의 사이즈)는 전체 평가 데이터셋 크기를 PART 수로 나눠주면 구할 수 있습니다.
    # 모델 학습할 때 배치 사이즈 구했던 것과 비슷합니다.
    chunk = len(customers)//NUM_PARTS
    if verbose != '':
        print(f'We will process {verbose} data as {NUM_PARTS} separate parts.')
        print(f'There will be {chunk} customers in each part (except the last part).')
        print('Below are number of rows in each part:')
    rows = []

    for k in range(NUM_PARTS):
        # 마지막 PART면 남은 부분을 모두 cc에 담습니다.
        if k==NUM_PARTS-1: 
            cc = customers[k*chunk:]
        # 마지막 PART가 아니면 앞에서부터 청크 단위로 잘라서 cc에 담습니다.
        else: 
            cc = customers[k*chunk:(k+1)*chunk]
        # 현재 PART에 포함된 customer_ID 수를 구하는 것을 통해 PART 사이즈를 계산합니다.
        s = test.loc[test.customer_ID.isin(cc)].shape[0]
        # rows에는 4개 PART의 크기가 담겨있습니다.
        rows.append(s)
    if verbose != '': print( rows )
    return rows, chunk



우리는 특정 customer가 지출액을 상환할 것인가가 궁금합니다. 테스트 데이터셋에는 customer_ID가 중복되어 들어가있기 때문에 중복은 제거하고 유니크한 customer_ID만 남겨둡니다. 그리고 데이터프레임 형식을 유지할 필요는 없기 때문에 1차원 데이터로 펼쳐주겠습니다.

In [44]:
customers = test[['customer_ID']].drop_duplicates().sort_index().values.flatten()
customers

이제 위에서 만든 함수로 customer_ID를 총 4개 그룹(PART)로 나누겠습니다.

In [48]:
# 테스트 데이터셋을 4개로 나눠주겠습니다.
NUM_PARTS = 4
rows,num_cust = get_rows(customers, test[['customer_ID']], NUM_PARTS = NUM_PARTS, verbose = 'test')

마지막 줄에 각 PART별 크기가 출력되었습니다. 모두 더했을 때 test 데이터셋의 사이즈와 같아야 합니다.

In [57]:
sum(rows) == len(test)

In [55]:
customers[0]

## Step 8. Infer Test

In [ ]:
# INFER TEST DATA IN PARTS
skip_rows = 0
skip_cust = 0
test_preds = []

for k in range(NUM_PARTS):
    
    # READ PART OF TEST DATA
    print(f'\nReading test data...')
    test = read_file(path = TEST_PATH)
    test = test.iloc[skip_rows:skip_rows+rows[k]]
    skip_rows += rows[k]
    print(f'=> Test part {k+1} has shape', test.shape )
    
    # PROCESS AND FEATURE ENGINEER PART OF TEST DATA
    test = process_and_feature_engineer(test)
    if k==NUM_PARTS-1: test = test.loc[customers[skip_cust:]]
    else: test = test.loc[customers[skip_cust:skip_cust+num_cust]]
    skip_cust += num_cust
    
    # TEST DATA FOR XGB
    X_test = test[FEATURES]
    dtest = xgb.DMatrix(data=X_test)
    test = test[['P_2_mean']] # reduce memory
    del X_test
    gc.collect()

    # INFER XGB MODELS ON TEST DATA
    model = xgb.Booster()
    model.load_model(f'XGB_v{VER}_fold0.xgb')
    preds = model.predict(dtest)
    for f in range(1,FOLDS):
        model.load_model(f'XGB_v{VER}_fold{f}.xgb')
        preds += model.predict(dtest)
    preds /= FOLDS
    test_preds.append(preds)

    # CLEAN MEMORY
    del dtest, model
    _ = gc.collect()

## Step 9. Create Submission CSV

In [ ]:
# WRITE SUBMISSION FILE
test_preds = np.concatenate(test_preds)
test = cudf.DataFrame(index=customers,data={'prediction':test_preds})
sub = cudf.read_csv('../input/amex-default-prediction/sample_submission.csv')[['customer_ID']]
sub['customer_ID_hash'] = sub['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
sub = sub.set_index('customer_ID_hash')
sub = sub.merge(test[['prediction']], left_index=True, right_index=True, how='left')
sub = sub.reset_index(drop=True)

# DISPLAY PREDICTIONS
sub.to_csv(f'submission_xgb_v{VER}.csv',index=False)
print('Submission file shape is', sub.shape )
sub.head()

In [ ]:
# PLOT PREDICTIONS
plt.hist(sub.to_pandas().prediction, bins=100)
plt.title('Test Predictions')
plt.show()